In [1]:
import cv2
import numpy as np
# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [2]:
def countVeh(img):
    height, width, channels = img.shape
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                if class_id != 0:
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #Visualizing CCTV Footage Vehicle Detection
    #font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            #cv2.putText(img, label, (x, y + 30), font, 3, color, 3)


    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return len(indexes)

In [3]:
def timeCount(cnt):
    cnt[cnt[:] < 3] = 3
    cnt[cnt[:] > 30 ] =30
    y = 3 * cnt
    stop_time = np.zeros((len(cnt),1))
    stop_time[0] = np.sum(y) - y[0]
    for i in range(1,len(cnt)):
        stop_time[i]=stop_time[i-1]-y[i]
    return stop_time,cnt,y

In [4]:
#Simple Examples Similar to a Cross-Section Representing CCTV Footage like Photos taken ON Indian Roads
c = np.zeros(4)
img1 = cv2.imread("test1.jfif")
img1 = cv2.resize(img1, None, fx=2, fy=2)
c[0] = countVeh(img1)
img2 = cv2.imread("test2.webp")
img2 = cv2.resize(img2, None, fx=1, fy=1)
c[1] = countVeh(img2)
img3 = cv2.imread("test3.jpg")
img3 = cv2.resize(img3, None, fx=0.8, fy=0.8)
c[2] = countVeh(img3)
img4 = cv2.imread("test4.jpg")
img4 = cv2.resize(img4, None, fx=0.6, fy=0.6)
c[3] = countVeh(img4)
stop_time,cnt,go_time = timeCount(c)
print("Vehicle Count \n" + str(cnt))
print("Green/Go Time \n" + str(go_time))
print("Stop Time \n" + str(stop_time))

Vehicle Count 
[22.  7. 16. 11.]
Green/Go Time 
[66. 21. 48. 33.]
Stop Time 
[[102.]
 [ 81.]
 [ 33.]
 [  0.]]
